# Notebook 2 - Defects / Features

This notebook demonstrates the library of different defects than can be introduced to a pipe in PipeFactory.

First we build a pipe standard pipe, this is a straight pipe of length 350.0 mm and has a radius of 70.0 and thickness of 3.0.

In [3]:
import numpy as np
import pipefactory as pf

Straight0 = {
    'Name': 'Straight0',
    'length': 350.0,
    'type': 'Straight',
}

section_list = [Straight0]

mesh = pf.Pipe(radius = 70., 
               thickness = 3.0, 
               section_list=section_list, 
               elem_type=("hex", False), 
               element_size = 6.0, 
               elements_through_thickness = 3)

mesh.export('foo.vtk')


There are two broad classes of defects that have been implemented. These involved one of two things

1. Displacing nodes, which include Dimple and Weld defects.

2. Removing elements, which includes involves holes and slits.

We will go through all types.

## Dimple Defects

These are defects which represent a dent (inwards). To define them we define a number of parameters

- $s_0$ (mm) the location along the length of the midline of the pipe.

- $\phi_0$ (degrees) the radial angle of the centre of the dimple.

- $A$ (mm) the amplitude of the dimple.

- $\ell$ (mm) lengthscale parameter controlling the extend of the dimple.

In [3]:
s0 = 150.0
phi0 = 45.0
A = 10.0 
ell = 25.0

mesh.add_defect_displacement(pf.Dimple(s0, phi0, A, ell))

mesh.export('foo_dimple.vtk')

Multiple defects can be added by simple calling the method '.add_defect_displacement()' more than once. Like this

In [3]:
mesh.add_defect_displacement(pf.Dimple(70.0, 90., A, ell))

mesh.export('foo_multiple_defects.vtk')

## Weld

The next defect / feature defines a weld as a thickening around the complete collar of the pipe. It is defined by

- $s_0$ (mm) the location along the length of the midline of the pipe.

- $A_{out}$ (degrees)  the amplitude of the dimple on the outside

- $A_{in}$ (mm) the amplitude of the dimple on the inside

- $r$ (mm) radius of the pipe

- $t$ (mm) thickness of the pipe

In [6]:
s0 = 150.0
Aout = 0.3
Ain = 0.1
ell_out = 6.
ell_in = 3.
radius = 70.
thickness = 3.


mesh_weld = pf.Pipe(radius = 70., 
               thickness = 3.0, 
               section_list=section_list, 
               elem_type=("hex", False), 
               element_size = 1.5, 
               elements_through_thickness = 5)

mesh_weld.add_defect_displacement(pf.Weld(s0, Aout, Ain, ell_out, ell_in, radius, thickness))

mesh_weld.export('foo_weld.vtk')

## Hole

To introduce a circular hole we called this time the method 'remove_elements()'. A hole is defed by three variables

- $s_0$ (mm) the location along the length of the midline of the pipe.

- $\phi_0$ (degrees) the radial angle of the centre of the dimple.

- $r$ (mm) the radius of the hole.

We start by building a fresh mesh. Then make and visualise the hole.

In [5]:
s0 = 100.0
phi0 = 45.0
radius = 20.0

mesh_with_hole = pf.Pipe(70.0, 3.0, [Straight0], ("hex", False), 2.0, 3)

mesh_with_hole.remove_elements(pf.Hole(s0, phi0, radius))

mesh_with_hole.export('foo_hole.vtk')

## Slits

In a simple way to a whole we can introduce a slit. The difference with this is we have introduced a slit profile. Which allows you to define the profile of the slit, so that not all elements are removed through thickness.

Naturally this collectively has more parameters to define the "slit".

- $s_0$ (mm) the location along the length of the midline of the pipe.

- $\phi_0$ (degrees) is the angle at the start of the slit

- $\phi_1$ (degrees) is the angle at the end of the slit

- $w$ (mm) is the slit width.

- 'partial' indicates if there is only a partial slit, i.e. not all elements through thickness are removed.

- 'profile' is a user-defined function which describes the profile of the slit through the thickness fo the slit.

So Let's do a simple slit first, for which all elements are removed.

In [4]:
mesh_with_slit = pf.Pipe(70.0, 3.0, [Straight0], ("hex", False), 2.0, 5)

mesh_with_slit.remove_elements(pf.Radial_Slit(s0=100.0, phi0=10., phi1=45.0, slit_width=3.0, radius=70.0, thickness=3.0, partial = False))

mesh_with_slit.export('foo_slit.vtk')

We can now build the same slit which has a partial profile in it.

In [17]:
mesh_with_slit = pf.Pipe(70.0, 3.0, [Straight0], ("hex", False), 2.0, 5)

class myprofile:
    def __init__(self):
        self.A = 2.0
    def __call__(self, ds, z):
        if z > 1. - self.A * np.exp(-12.0*(ds-0.5)**2):
            return True
        else:
            return False

mesh_with_slit.remove_elements(pf.Radial_Slit(s0=200.0, phi0=10., phi1=45.0, slit_width=3.0, radius = 70., thickness = 3.0, partial = True, profile=myprofile()))

mesh_with_slit.export('foo_partial_slit.vtk')